# Filtrado colaborativo usuario a usuario

**Autor**: Arturo Sánchez Palacio

Basado en: https://github.com/lazyprogrammer

**Fecha de última revisión: 16/I/2020**

Este notebook se emplearán los siguientes módulos:

In [ ]:
import os
import pickle
import numpy as np

Comenzamos cargando los datos almacenados en los diccionarios generados en el preprocesamiento:

__Nota.__ Si el notebook de preprocesamiento no se ha ejecutado antes debe ejecutarse ahora para generar los datos con los que se trabaja en este notebook:

In [ ]:
with open('./data/usuario_pelicula.json', 'rb') as f:
  usuario_pelicula = pickle.load(f)

with open('./data/pelicula_usuario.json', 'rb') as f:
  pelicula_usuario = pickle.load(f)

with open('./data/usuariopeli_rating.json', 'rb') as f:
  usuariopeli_rating = pickle.load(f)

with open('./data/usuariopeli_rating_test.json', 'rb') as f:
  usuariopeli_rating_test = pickle.load(f)

De nuevo vamos a cálcular N y M como el número máximo de usuarios películas. Esta vez es algo más complicado pues puede haber películas que no aparezcan en el training set pero sí en el test set. Esto mismo es muy improbable para usuarios por lo que N se puede calcular de manera trivial:

#### To do. Calcular el número de usuarios:

Para calcular M buscamos el máximo tanto en pelicula_usuario como en usuariopeli_rating_test y fijamos como M el máximo de ambas:

#### To do calcular el número de películas:

In [ ]:
print(M,N)

Aquí reflexionamos de nuevo sobre el coste computacional de la operación. Recordemos que el algoritmo es  $O(N^2 * M)$ así que para valores de N muy altos deberíamos plantearnos volver a muestrear. 10.000 es elevado pero está bien así que pododemos continuar:

__Nota.__ Recordemos las fórmulas que definen los pesos y las predicciones:

![title](media/pesos.png)

![title](media/predicciones.png)

Como discutimos previamente en la explicación teórica fijamos el número de vecinos máximo que vamos a considerar y un umbral de películas comunes (no calcularemos pesos para usuarios que tengan menos películas en común que este umbral). Fijamos explorar 25 vecinos (como máximo) y un umbral de al menos cinco películas en común con el usuario:

In [ ]:
K = 25 #número máximo de vecinos
umbral = 5 #número mínimo de películas comunes

Construimos tres listas vacías en las que almacenaremos:

In [ ]:
vecinos = [] #vecinos para el usuario
medias = [] # valoración media de cada usuario
desviaciones = [] #desviaciones de cada usuario

Para cada usuario:

In [ ]:
from sortedcontainers import SortedList
for i in range(N):
    #buscamos los K usuarios más próximos a i
    peliculas_i = usuario_pelicula[i] #pelis que el usuario ha visto
    peliculas_i_set = set(peliculas_i) #las almaceno en un conjunto
    ratings_i = { pelicula:usuariopeli_rating[(i, pelicula)] for pelicula in peliculas_i } #construyo un diccionario de clave película y valor sus puntaciones
    avg_i = np.mean(list(ratings_i.values())) #puntuación media del usuario i
    dev_i = {pelicula:(rating - avg_i) for pelicula, rating in ratings_i.items()} #desviaciones del usuario i
    dev_i_values = np.array(list(dev_i.values())) #array de Numpy con las desviaciones
    sigma_i = np.sqrt(dev_i_values.dot(dev_i_values)) #denominador del coeficiente de Pearson

    medias.append(avg_i) #guardo las medias para usarlas en un futuro
    desviaciones.append(dev_i) #guardo las desviaciones para usarlas en un futuro

    sl = SortedList() #Creo una lista ordenada en la que almacenar los pesos que ya tengo calculados (mantengo las 25 entradas que nos interesan)
                         
    for j in range(N): #Para cada usuario distinto de i calculo el peso i j (no es eficiente computacionalmente pero costaría más memoria optimizarlo)
    
        if j != i: #el propio usuario no puede ser su propio vecino
            peliculas_j = usuario_pelicula[j]
            peliculas_j_set = set(peliculas_j)
            peliculas_comunes = (peliculas_i_set & peliculas_j_set) # peliculas comunes entre i y j (intersección)
            if len(peliculas_comunes) > umbral: #solo calculamos el peso si son suficientemente similares
        
                ratings_j = { pelicula:usuariopeli_rating[(j, pelicula)] for pelicula in peliculas_j } #diccionario
                avg_j = np.mean(list(ratings_j.values())) #puntuación media
                dev_j = { pelicula:(rating - avg_j) for pelicula, rating in ratings_j.items() } #desviaciones
                dev_j_values = np.array(list(dev_j.values())) #array de Numpy con las desviaciones
                sigma_j = np.sqrt(dev_j_values.dot(dev_j_values)) #denominador del coeficiente de Pearson

                # calculate correlation coefficient
                numerador = sum(dev_i[m]*dev_j[m] for m in peliculas_comunes)
                w_ij = numerador / (sigma_i * sigma_j)

                sl.add((-w_ij, j)) #añadimos el peso y el usuario a la lista ordenada (- porque ordena ascendentemente)
                if len(sl) > K: # Si la lista supera el límite de vecinos eliminamos el peso menos importante
                    del sl[-1]

    
    vecinos.append(sl) #almacenamos los vecinos (usuario 0 en posición 0...)
    print(i,"/",N)

A continuación construimos una función para calcular las predicciones. Partiendo de un usuario i y una película m predice la puntuación que este usuario daría a la película:

In [ ]:
N

In [ ]:
def predict(i, m):
   
    numerador = 0 #suma del producto de los pesos y las desviaciones
    denominador = 0 #suma de los valores absolutos de los pesos
    for vecino in vecinos[i]:
        neg_w =vecino[0]
        j = vecino[1]
        desviaciones_j = desviaciones[j]
        try:
        # Si el vecino ha valorado la película calculamos:
        ###EJERCICIO EMPIEZA
            numerador +=  #Recordar que almacenamos el peso en negativo
            denominador += 
        ###EJERCICIO ACABA
        except KeyError:
        #El vecino puede no haber valorado la película que predecimos.
        #Lanzaremos una excepción cuando buscamos en el diccionario y no encontramos la valoración.
            pass

    if denominador == 0:
        prediccion = medias[i] #No podemos hacer nada mejor que esto
    else:
        prediccion = numerador / denominador + medias[i]
    ###EJERCICIO EMPIEZA
     # la fórmula no está acotada así que la acotamos manualmente (esto es peligroso)
     # la fórmula no está acotada así que la acotamos manualmente (esto es peligroso)
    ###EJERCICIO ACABA
    return prediccion

Creamos dos listas para almacenar las predicciones y etiquetas:

In [ ]:
predicciones_train = []
etiquetas_train = []

#### To do. Realizamos las predicciones para cada película y usuario en entrenamiento:

In [ ]:
for (i, m), etiqueta in usuariopeli_rating.items():
     #Calculamos la predicción para la película

    # Almacenamos la predicción y la etiqueta




#### To do. Análogamente en el test:

#### To do. Implementamos una función trivial para calcular el error medio cuadrático:

In [ ]:
def mse(p, t):

Podemos observar el error final:

In [ ]:
print('train mse:', mse(predicciones_train, etiquetas_train))
print('test mse:', mse(predicciones_test, etiquetas_test))

## Conclusión

Para ser una aproximación tan naïve los resultados no son nada despreciables. Es poco eficaz computacionalmente como hemos podido observar. Sin embargo es interesante comparar este resultado con los que obtendremos más adelante.